In [1]:
import stripe
import datetime
import os
from typing import List

# Set API Key as Environment Variable
On a mac, go to ~/.zshrc and EXPORT STRIPE_TEST_API_KEY=whateverthekeyis<br>
Then either reboot or type source ~/.zshrc

In Windows, refer to this site: https://www.computerhope.com/issues/ch000549.htm#windows11<br>
You might need to reboot.

In [2]:
stripe.api_key = os.environ.get("STRIPE_TEST_API_KEY")

# Specify Date Range

In [3]:
START_DATE = 'Mar 30 2022'
END_DATE = 'Aug 10 2022'

# General Utility Functions

In [4]:
def convert_epoch_to_timestamp(epoch: int) -> datetime.datetime:
    """
    Converts timestamp, provided as the epoch, 
    from stripe to a python datetime object
    """
    return datetime.datetime.fromtimestamp(epoch)

def convert_timestamp_to_epoch(date: str) -> int:
    """
    Converts date string in the form of Mar 30 2022 (%b %d %Y)
    to number of seconds since epoch. Date is in UTC time, so
    Mar 30 2022 passed in as an argument would be returned as 
    0000 UTC on Mar 30 2022.
    """
    dt_obj = datetime.datetime.strptime(date, '%b %d %Y')
    timestamp = dt_obj.replace(tzinfo=datetime.timezone.utc).timestamp()
    return int(timestamp)

def convert_timestamp_to_friendly(dt_obj: datetime.datetime) -> datetime.datetime:
    """
    Converts python datetime object to a
    friendly date-time that works with Allegiance
    """
    return dt_obj.strftime('%b %-d, %Y, %-I:%M:%S %p')

def convert_cents_to_dollars(cents: int) -> int:
    """
    Converts cents to USD with appropriate number of decimal points
    """
    return "{:.2f}".format(cents / 100)

def combine_card_expiration_month_year(card_expiration_month: List[int], 
                                       card_expiration_year: List[int]) -> List[str]:
    """
    Combines credit card expiration month and year
    """
    card_expiration = []
    for month in card_expiration_month:
        for year in card_expiration_year:
            card_expiration.append(str(month) + '/' + str(year))
    return card_expiration

def retrieve_customer_name(customer_id: str) -> str:
    """
    Returns the full name of the customer based on the Stripe Customer ID
    If no customer ID is present, return NONAME
    """
    if customer_id is None:
        return 'NONAME'
    
    customer_obj = stripe.Customer.retrieve(customer_id)
    return customer_obj['name']

# Retrieve Required Objects from Stripe

In [5]:
charges = stripe.Charge.list(
    created={
        'gt': convert_timestamp_to_epoch(START_DATE),
        'lt': convert_timestamp_to_epoch(END_DATE)
    }
)
payouts = stripe.Payout.list(
    created={
        'gt': convert_timestamp_to_epoch(START_DATE),
        'lt': convert_timestamp_to_epoch(END_DATE)
    }
)
balance_transactions = stripe.BalanceTransaction.list(limit=3)

# Retrieve Dates
These values are dates/times of the transaction

In [6]:
dates = [convert_epoch_to_timestamp(charge['created']) for charge in charges['data']]
friendly = [convert_timestamp_to_friendly(date) for date in dates]

# Retrieve Amounts Charged/Donated

In [7]:
amounts = [convert_cents_to_dollars(charge['amount']) for charge in charges['data']]

# Retrieve Payment Intent
This value (prefixed by "pi_" replaces Verisign Key)

In [8]:
intent = [charge['payment_intent'] for charge in charges['data']]

# Retrieve Credit Card Data

In [9]:
card_brands = [charge['source']['card']['brand'] for charge in charges['data']]
last_fours = [charge['source']['card']['last4'] for charge in charges['data']]
card_expiration_months = [charge['source']['card']['exp_month'] for charge in charges['data']]
card_expiration_years = [charge['source']['card']['exp_year'] for charge in charges['data']]
card_expirations = combine_card_expiration_month_year(card_expiration_months, card_expiration_years)

# Retrieve Customer Data based on the ID of Transaction
This value is prefixed by "cus_". If no ID is present, customer=None

In [10]:
customer_ids = [charge['customer'] for charge in charges['data']]
customer_names = [retrieve_customer_name(customer_id) for customer_id in customer_ids]

# Stripe Payout ID
This value (prefixed by "po_") will be used for the batch number

In [11]:
payout_id = [print(payout['id']) for payout in payouts['data']]

po_1LSWc2F2quY7yoKLUIrASHCG


# Output Data / Final Results
Note to self: I need to include refunds. How do those presently appear in PayPal report?

In [12]:
results = list(zip(intent, friendly, card_brands, last_fours, amounts, 
                   card_expirations, amounts, customer_names))

print(*results, sep='\n')

('pi_3Kj8PAF2quY7yoKL0i4IgUOt', 'Mar 30, 2022, 4:46:13 PM', 'Discover', '1117', '1.00', '3/2026', '1.00', 'Mickey Mouse')
('pi_3Kj8FtF2quY7yoKL1QWXjNkE', 'Mar 30, 2022, 4:36:37 PM', 'American Express', '0005', '3.00', '3/2024', '3.00', 'NONAME')
('pi_3Kj8DTF2quY7yoKL1HLhuyQc', 'Mar 30, 2022, 4:34:07 PM', 'MasterCard', '4444', '2.00', '3/2023', '2.00', 'NONAME')
